# Corporate Deposits Seasonality Model with PyMC

In [ ]:
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from validmind.datasets.regression import fred_deposits as demo_dataset

deposits_df, deposits_seasonality_df, fedfunds_df, tb3ms_df, gs10_df, gs30_df = demo_dataset.load_data()

df = deposits_seasonality_df.copy()

df["Month"] = df.index

target_column = demo_dataset.target_column

In [ ]:
fig = px.line(df, x=df["Month"], y=target_column, title='Original Data')
fig.update_layout(xaxis_title='Month', yaxis_title=target_column)
fig.show()

In [ ]:
t = (df["Month"]- pd.Timestamp("1900-01-01")).dt.days.to_numpy()
t_min = np.min(t)
t_max = np.max(t)
t = (t - t_min) / (t_max - t_min)

In [ ]:
y = df[target_column].to_numpy()
y_max = np.max(y)
y = y / y_max

In [ ]:
with pm.Model(check_bounds=False) as linear:
    alpha = pm.Normal("alpha", mu=0, sigma=5)
    beta = pm.Normal("beta", mu=0, sigma=5)
    sigma = pm.HalfNormal("sigma", sigma=5)
    trend = pm.Deterministic("trend", alpha + beta * t)
    pm.Normal("likelihood", mu=trend, sigma=sigma, observed=y)

    linear_prior = pm.sample_prior_predictive()

In [ ]:
likelihood = az.extract(linear_prior, group="prior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_prior, group="prior", num_samples=100)["trend"] * y_max

In [ ]:
# Plotting the prior likelihood
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

In [ ]:
with pm.Model(check_bounds=False) as linear:
    alpha = pm.Normal("alpha", mu=0, sigma=0.5)
    beta = pm.Normal("beta", mu=0, sigma=0.5)
    sigma = pm.HalfNormal("sigma", sigma=0.5)
    trend = pm.Deterministic("trend", alpha + beta * t)
    pm.Normal("likelihood", mu=trend, sigma=sigma, observed=y)

    linear_prior = pm.sample_prior_predictive()

In [ ]:
likelihood = az.extract(linear_prior, group="prior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_prior, group="prior", num_samples=100)["trend"] * y_max

In [ ]:
# Plotting the prior likelihood
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

In [ ]:
with linear:
    linear_trace = pm.sample(return_inferencedata=True)
    linear_prior = pm.sample_posterior_predictive(trace=linear_trace)

In [ ]:
likelihood = az.extract(linear_prior, group="posterior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_trace, group="posterior", num_samples=100)["trend"] * y_max

In [ ]:
# Plotting the prior likelihood
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Posterior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Posterior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

In [ ]:
n_order = 10
periods = (df["Month"] - pd.Timestamp("1900-01-01")).dt.days / 365.25

fourier_features = pd.DataFrame(
    {
        f"{func}_order_{order}": getattr(np, func)(2 * np.pi * periods * order)
        for order in range(1, n_order + 1)
        for func in ("sin", "cos")
    }
)
fourier_features

In [ ]:
coords = {"fourier_features": np.arange(2 * n_order)}
with pm.Model(check_bounds=False, coords=coords) as linear_with_seasonality:
    alpha = pm.Normal("alpha", mu=0, sigma=0.5)
    beta = pm.Normal("beta", mu=0, sigma=0.5)
    sigma = pm.HalfNormal("sigma", sigma=0.1)
    beta_fourier = pm.Normal("beta_fourier", mu=0, sigma=10, dims="fourier_features")
    seasonality = pm.Deterministic(
        "seasonality", pm.math.dot(beta_fourier, fourier_features.to_numpy().T)
    )
    trend = pm.Deterministic("trend", alpha + beta * t)
    mu = trend + seasonality
    pm.Normal("likelihood", mu=mu, sigma=sigma, observed=y)

    linear_seasonality_prior = pm.sample_prior_predictive()

In [ ]:
likelihood = az.extract(linear_seasonality_prior, group="prior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_seasonality_prior, group="prior", num_samples=100)["trend"] * y_max
seasonality = az.extract(linear_seasonality_prior, group="prior", num_samples=100)["seasonality"] * 100

In [ ]:
# Plotting the prior likelihood
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

# Plotting the seasonality lines
fig = go.Figure()

for sample in seasonality.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.update_layout(title="Prior Seasonality Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

In [ ]:
coords = {"fourier_features": np.arange(2 * n_order)}
with pm.Model(check_bounds=False, coords=coords) as linear_with_seasonality:
    alpha = pm.Normal("alpha", mu=0, sigma=0.5)
    beta = pm.Normal("beta", mu=0, sigma=0.5)
    sigma = pm.HalfNormal("sigma", sigma=0.1)
    beta_fourier = pm.Normal("beta_fourier", mu=0, sigma=0.1, dims="fourier_features")
    seasonality = pm.Deterministic(
        "seasonality", pm.math.dot(beta_fourier, fourier_features.to_numpy().T)
    )
    trend = pm.Deterministic("trend", alpha + beta * t)
    mu = trend  + seasonality
    pm.Normal("likelihood", mu=mu, sigma=sigma, observed=y)

    linear_seasonality_prior = pm.sample_prior_predictive()

In [ ]:
likelihood = az.extract(linear_seasonality_prior, group="prior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_seasonality_prior, group="prior", num_samples=100)["trend"] * y_max
seasonality = az.extract(linear_seasonality_prior, group="prior", num_samples=100)["seasonality"] * 100

In [ ]:
# Plotting the prior likelihood
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Prior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

# Plotting the seasonality lines
fig = go.Figure()

for sample in seasonality.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.update_layout(title="Prior Seasonality Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()

In [ ]:
with linear_with_seasonality:
    linear_seasonality_trace = pm.sample(return_inferencedata=True)
    linear_seasonality_posterior = pm.sample_posterior_predictive(trace=linear_seasonality_trace)


In [ ]:
likelihood = az.extract(linear_seasonality_posterior, group="posterior_predictive", num_samples=100)["likelihood"] * y_max
trend = az.extract(linear_trace, group="posterior", num_samples=100)["trend"] * y_max
seasonality = az.extract(linear_seasonality_trace, group="posterior", num_samples=100)["seasonality"] * 10000

In [ ]:
# Plotting the posterior predictive
fig = go.Figure()

for sample in likelihood.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Posterior Predictive", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the posterior trend lines
fig = go.Figure()

for sample in trend.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.add_trace(go.Scatter(x=df['Month'], y=df[target_column], mode='lines+markers', marker=dict(color='black', size=5), name='Data'))

fig.update_layout(title="Posterior Trend Lines", xaxis_title="Date", yaxis_title=target_column)

fig.show()


# Plotting the seasonality lines
fig = go.Figure()

for sample in seasonality.T:
    fig.add_trace(go.Scatter(x=df['Month'], y=sample, mode='lines', line=dict(color='blue', width=1), opacity=0.05))

fig.update_layout(title="Posterior Seasonality", xaxis_title="Date", yaxis_title=target_column)

fig.show()